In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.3.0'

In [3]:
print "Author: Tommy Mulc"

Author: Tommy Mulc


Create a TensorFlow cluster with one worker node and one ps node.

In [4]:
cluster_spec = tf.train.ClusterSpec({'worker' : ['localhost:2223'], 'ps' : ['localhost:2222']})
server = tf.train.Server(cluster_spec,job_name='worker')

**Now launch run all the cells in the parameter server notebook**

Create variables locally then makes global copy.  One worker scenario

In [5]:
tf.reset_default_graph()

In [6]:
#create local graph like normal specifying the local device
with tf.device('/job:worker/task:0'):
    a = tf.Variable([0.],name='a',collections=[tf.GraphKeys.LOCAL_VARIABLES])
    b = tf.constant([100.])
    loss = tf.abs(a-b)
    
    optimizer = tf.train.GradientDescentOptimizer(.1)
    grads,local_vars = zip(*optimizer.compute_gradients(loss,var_list=tf.local_variables()))
    local_update = optimizer.apply_gradients(zip(grads,local_vars))
    
    
    init_local = tf.local_variables_initializer()

#create the globabl copies on the ps
with tf.device('/job:ps/task:0'):
    for v in tf.local_variables():
        v_g = tf.get_variable('g/'+v.op.name,
                            shape = v.shape,
                            dtype = v.dtype,
                            trainable=True,
                            collections=[tf.GraphKeys.GLOBAL_VARIABLES,tf.GraphKeys.TRAINABLE_VARIABLES])


#gloabl updates
with tf.device('/job:worker/task:0'):
    #this needs to be updated.  Clearly not robust for any graph more complext
    global_vars = tf.global_variables()
    global_update = optimizer.apply_gradients(zip(grads,global_vars))

#create init op on the chief node
with tf.device('/job:worker/task:0'):
    init_global = tf.global_variables_initializer()

View the device placement of ops and variables

In [7]:
a_global = tf.global_variables()[0]

In [10]:
print(a.device)
print(b.device)
print(loss.device)
print(local_update.device)
print(global_update.device)
print(init_global.device)
print(init_local.device)
print(a_global.device)

/job:worker/task:0
/job:worker/task:0
/job:worker/task:0
/job:worker/task:0
/job:ps/task:0
/job:ps/task:0
/job:worker/task:0
/job:ps/task:0


Now, let's view the states of local and global variables as we do local then global updates

In [11]:
sess = tf.Session(target=server.target)
sess.run([init_local,init_global])

[None, None]

In [12]:
sess.run([a,a_global])

[array([ 0.], dtype=float32), array([-1.26032162], dtype=float32)]

In [13]:
sess.run(local_update)

In [14]:
sess.run([a,a_global])

[array([ 0.1], dtype=float32), array([-1.26032162], dtype=float32)]

Notice that the state of the global variable hasn't changed

In [15]:
sess.run(global_update)

In [16]:
sess.run([a,a_global])

[array([ 0.1], dtype=float32), array([-1.16032159], dtype=float32)]